In [4]:
import ee
import colorcet
from matplotlib.cm import get_cmap
import geemap

ee.Initialize(project='hotspotstoplight')

In [5]:
bbox = ee.Geometry.Polygon( # all of costa rica
    [[
        [-85.9, 8.0],  # Lower left corner (southwest)
        [-85.9, 11.2], # Upper left corner (northwest)
        [-82.5, 11.2], # Upper right corner (northeast)
        [-82.5, 8.0],  # Lower right corner (southeast)
        [-85.9, 8.0]   # Closing the polygon by repeating the first point
    ]]
)

In [6]:
URL = 'gs://hotspotstoplight_floodmapping/data/costa_rica/outputs/flood_prob.tif'
image = geemap.load_GeoTIFF(URL)

In [11]:
def z_score_normalize(image, bbox):
    mean = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=bbox, scale=30, maxPixels=1e13).getNumber(image.bandNames().get(0))
    std_dev = image.reduceRegion(reducer=ee.Reducer.stdDev(), geometry=bbox, scale=30, maxPixels=1e13).getNumber(image.bandNames().get(0))
    return image.subtract(mean).divide(std_dev)

def make_vulnerability_data(bbox):
    print("Generating vulnerability data...")

    gdp_per_capita = ee.Image("projects/sat-io/open-datasets/GRIDDED_HDI_GDP/GDP_per_capita_PPP_1990_2015_v2").select('b1').clip(bbox)
    hdi_30arc = ee.Image("projects/sat-io/open-datasets/GRIDDED_HDI_GDP/HDI_1990_2015_v2").select('b1').clip(bbox)
    population = ee.ImageCollection("projects/sat-io/open-datasets/hrsl/hrslpop").median().clip(bbox)

    # First, multiply GDP by HDI and then Z-score normalize the results
    gdp_hdi_product = gdp_per_capita.multiply(hdi_30arc)
    gdp_hdi_product_normalized = z_score_normalize(gdp_hdi_product, bbox)

    # Second, Z-score normalize population
    population_normalized = z_score_normalize(population, bbox)

    # Then, divide population by the Z-scored product of GDP and HDI
    vuln = population_normalized.divide(gdp_hdi_product_normalized).rename('vuln')

    return vuln


In [12]:
vuln = make_vulnerability_data(bbox)

Generating vulnerability data...


In [13]:
vizParamsProbability = {
    'min': 0,
    'max': 1,
    'palette': colorcet.bky[::-1]
}

viz_params_pop = {
    'min': 0,
    'max': 1,
    'palette': colorcet.gwv
}

viz_params_vuln = {
    'min': -7,
    'max': 17,
    'palette': colorcet.bmy
}


Map = geemap.Map()
Map.add("layer_manager")
Map.add("basemap_selector")

geometry = image.geometry()
Map.centerObject(geometry, 8)
Map.addLayer(image, vizParamsProbability, 'Flood Probability')
# Map.addLayer(exp, viz_params_pop, "Exposure Data")
Map.addLayer(vuln, viz_params_vuln, "Vulnerability Data")
Map

Map(center=[8.354477700025758, -84.82327521031193], controls=(WidgetControl(options=['position', 'transparent_…